2025-10-16 22:21:24 Thursday




### 1.请你解释一下大语言模型（LLM）中的 Self-Attention 机制是如何工作的？

Self-Attention 通过 QK 相似度衡量全局依赖，用 softmax 得到动态权重，进而加权组合所有 token 的信息，实现了全局信息交互和长程依赖建模。

### **2.在大模型训练中，Layer Normalization（LayerNorm） 和 Batch Normalization（BN） 有什么区别？**

他们都是计算均值和方差，再以均值和方差计算归一化，然后再做可学习的可学习仿射变换，这样可以使的每次分布相近，训练收敛稳定,但归一化维度不同。
$$
\hat{x} = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}}, \quad y = \gamma \hat{x} + \beta
$$

- BN：在 batch 维度上对每个特征归一化，依赖于 batch size 的统计；
- LN：在特征维度上对每个样本独立归一化，不依赖 batch。

在 NLP 或 Transformer 中，因为每个样本序列长度不同、batch 小且不稳定，BN 统计会抖动；而 LN 对单个样本操作，更稳定，适合可变长度输入。
因此 Transformer 使用 LayerNorm 来保证每个 token 的特征尺度一致，避免依赖 batch 统计量。

### **3. 为什么 Transformer 中使用 Pre-LN 而不是 Post-LN？**

**Post-LN** 是先加残差再归一化：

$$y = \text{LayerNorm}(x + F(x))$$
**Pre-LN** 是先归一化再进入子层，然后再加残差：

$$y = x + F(\text{LayerNorm}(x))$$

在 **Post-LN** 结构中，LayerNorm 位于残差 **之后**，反向传播时梯度必须先经过 LayerNorm 才能流向前面的网络。
这会导致梯度在层间被不断缩放，随着网络加深，梯度逐渐衰减或放大，导致**训练不稳定、难以收敛**。
这一问题在原始 6 层 Transformer 中尚可接受，但在数十层甚至上百层的大模型中会非常明显。

改为 **Pre-LN** 后，归一化放在子层输入端，梯度可以沿残差路径直接传递，**显著提升了深层网络的稳定性和收敛速度**。
因此，GPT、BERT、T5 等现代大模型普遍采用 Pre-LN 结构。

不过，Pre-LN 也有一定代价：

1. **收敛到较差的最优点**：归一化在前使得梯度方向更保守，可能限制模型表达；
2. **推理时输出未被归一化**，浅层模型在测试时表现可能略差。




### 4.**在大模型训练中常见的优化器 Adam 是如何工作的？请你解释它与 SGD 的区别，并说明「一阶动量」「二阶动量」各代表什么含义。**

**Adam（Adaptive Moment Estimation）** 结合了动量法（Momentum）和 RMSProp 的优点。
对于参数 $\theta$，每一步更新公式如下：

$$
m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t \quad \text{（一阶动量：梯度的指数加权平均）}
$$

$$
v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2 \quad \text{（二阶动量：梯度平方的指数加权平均）}
$$

进行偏差校正：

$$
\hat{m}_t = \frac{m_t}{1-\beta_1^t}, \quad
\hat{v}_t = \frac{v_t}{1-\beta_2^t}
$$

最终更新参数：

$$
\theta_t = \theta_{t-1} - \alpha \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}
$$


#### **直观理解**

* **一阶动量**：平滑梯度方向，起到“惯性”作用，使下降更稳定；
* **二阶动量**：自适应地调整学习率，大梯度维度下降慢，小梯度维度下降快；
* 因此 Adam 能自动调节步长，避免震荡并加快收敛。


#### **与 SGD 的区别**

* **SGD（随机梯度下降）** 使用固定学习率，只依赖当前梯度；
* **Adam** 利用历史梯度的指数加权平均（动量）与平方平均（方差估计）进行自适应更新，使训练更加稳定且对学习率不敏感。


### 5.大模型训练中常用的 混合精度训练（Mixed Precision Training） 是什么？


在大模型训练中，混合精度训练（Mixed Precision Training, MPT）通过在保持数值精度的前提下，降低显存占用并提升计算效率。

训练中涉及三类参数：模型参数（weights）、梯度（gradients）、优化器状态（optimizer states）。
混合精度训练的策略是：

- 大部分计算与存储使用 半精度（FP16），显著降低显存需求并加速运算；

- 对关键变量（如优化器状态、权重 master copy）使用 全精度（FP32），以防止数值上溢或下溢。

主要风险是 数值不稳定（overflow / underflow），可能导致 NaN。
为此通常采用 损失缩放（Loss Scaling） 技术：
将 loss 乘以放大系数 $S$ 后再反向传播，最后再除以 $S$，以避免梯度在 FP16 范围下消失。

这种方法在保持精度的同时显著减少显存消耗，是当前大模型训练的标准做法。